# Collaboration and Competition

---

In this notebook, you will learn how to use the Unity ML-Agents environment for the third project of the [Deep Reinforcement Learning Nanodegree](https://www.udacity.com/course/deep-reinforcement-learning-nanodegree--nd893) program.

### 1. Start the Environment

We begin by importing the necessary packages.  If the code cell below returns an error, please revisit the project instructions to double-check that you have installed [Unity ML-Agents](https://github.com/Unity-Technologies/ml-agents/blob/master/docs/Installation.md) and [NumPy](http://www.numpy.org/).

In [ ]:
from unityagents import UnityEnvironment
import numpy as np

Next, we will start the environment!  **_Before running the code cell below_**, change the `file_name` parameter to match the location of the Unity environment that you downloaded.

- **Mac**: `"path/to/Tennis.app"`
- **Windows** (x86): `"path/to/Tennis_Windows_x86/Tennis.exe"`
- **Windows** (x86_64): `"path/to/Tennis_Windows_x86_64/Tennis.exe"`
- **Linux** (x86): `"path/to/Tennis_Linux/Tennis.x86"`
- **Linux** (x86_64): `"path/to/Tennis_Linux/Tennis.x86_64"`
- **Linux** (x86, headless): `"path/to/Tennis_Linux_NoVis/Tennis.x86"`
- **Linux** (x86_64, headless): `"path/to/Tennis_Linux_NoVis/Tennis.x86_64"`

For instance, if you are using a Mac, then you downloaded `Tennis.app`.  If this file is in the same folder as the notebook, then the line below should appear as follows:
```
env = UnityEnvironment(file_name="Tennis.app")
```

In [ ]:
env = UnityEnvironment(file_name="./Tennis_Linux/Tennis.x86_64", no_graphics=True)

Environments contain **_brains_** which are responsible for deciding the actions of their associated agents. Here we check for the first brain available, and set it as the default brain we will be controlling from Python.

In [ ]:
# get the default brain
brain_name = env.brain_names[0]
brain = env.brains[brain_name]

### 2. Examine the State and Action Spaces

In this environment, two agents control rackets to bounce a ball over a net. If an agent hits the ball over the net, it receives a reward of +0.1.  If an agent lets a ball hit the ground or hits the ball out of bounds, it receives a reward of -0.01.  Thus, the goal of each agent is to keep the ball in play.

The observation space consists of 8 variables corresponding to the position and velocity of the ball and racket. Two continuous actions are available, corresponding to movement toward (or away from) the net, and jumping. 

Run the code cell below to print some information about the environment.

In [ ]:
# reset the environment
env_info = env.reset(train_mode=True)[brain_name]

# number of agents 
num_agents = len(env_info.agents)
print('Number of agents:', num_agents)

# size of each action
action_size = brain.vector_action_space_size
print('Size of each action:', action_size)

# examine the state space 
states = env_info.vector_observations
state_size = states.shape[1]
print('There are {} agents. Each observes a state with length: {}'.format(states.shape[0], state_size))
print('The state for the first agent looks like:', states[0])

### 3. Take Random Actions in the Environment

In the next code cell, you will learn how to use the Python API to control the agents and receive feedback from the environment.

Once this cell is executed, you will watch the agents' performance, if they select actions at random with each time step.  A window should pop up that allows you to observe the agents.

Of course, as part of the project, you'll have to change the code so that the agents are able to use their experiences to gradually choose better actions when interacting with the environment!

### 4. It's Your Turn!

Now it's your turn to train your own agent to solve the environment!  When training the environment, set `train_mode=True`, so that the line for resetting the environment looks like the following:
```python
env_info = env.reset(train_mode=True)[brain_name]
```

In [ ]:
import torch
from collections import namedtuple, deque
import random
import matplotlib.pyplot as plt
import pandas as pd

In [ ]:
from maddpg_agents import Maddpg

In [ ]:
rand_seed = 10                     
n_episodes = 100000               
max_t = 10000                      

init_noise = 1.0                   
noise_reduction = 0.995            

In [ ]:
def ma_ddpg_train(n_episodes=10000,max_t=3000,print_every=100):
    
    scores_deque = deque(maxlen=print_every)
    all_scores = []
    
    noise = init_noise
    maddpg = Maddpg(state_size, action_size, num_agents, rand_seed)

    for i_episode in range(n_episodes+1):  
             
        env_info = env.reset(train_mode=True)[brain_name]     
        maddpg.reset()                                             
        states = env_info.vector_observations                      
        scores = np.zeros(num_agents)                              

        for steps in range(max_t):
            actions = maddpg.act(states, noise)               
            noise *= noise_reduction                              
            env_info = env.step(actions)[brain_name]             
            next_states = env_info.vector_observations           
            rewards = env_info.rewards                         
            dones = env_info.local_done                          
            
            maddpg.step(states, actions, rewards, next_states, dones, i_episode)  

            scores += rewards                             
            states = next_states
            if np.any(dones):                             
                break
        
        all_scores.append(np.max(scores))
        scores_deque.append(np.max(scores))
        avg_score = np.mean(scores_deque)

        # Display statistics 
        print('\rEpisode {}\tAverage Score: {:.2f}\tEpisode score (max over agents): {:.2f}'.format(i_episode, avg_score, np.max(scores)), end="")
        if (i_episode > 99) and (avg_score >=0.5):
            print('\rEnvironment solved in {} episodes with an Average Score of {:.2f}'.format(i_episode, avg_score))
            
            for idx, agent in enumerate(maddpg.agents):
                actor_local_filename = './actor_local_' + str(idx) + '_' + str(i_episode) + '.pth'
                critic_local_filename = './critic_local_' + str(idx) + '_' + str(i_episode) + '.pth'           
                actor_target_filename = './actor_target_' + str(idx) + '_' + str(i_episode) + '.pth'
                critic_target_filename = './critic_target_' + str(idx) + '_' + str(i_episode) + '.pth'            

                torch.save(agent.actor_local.state_dict(), actor_local_filename) 
                torch.save(agent.critic_local.state_dict(), critic_local_filename)             
                torch.save(agent.actor_target.state_dict(), actor_target_filename) 
                torch.save(agent.critic_target.state_dict(), critic_target_filename)

                return all_scores

    return all_scores

In [ ]:
def plot_training(scores):
    fig = plt.figure()
    ax = fig.add_subplot(111)
    plt.ylabel('Score')
    plt.xlabel('Episode #')
    plt.show()

In [ ]:
# Launch training
scores = ma_ddpg_train()
plot_training(scores)

In [ ]:
env.close()